<a href="https://colab.research.google.com/github/GunjanRoy4224/CV-and-NLP/blob/main/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NLP Track Capstone Project: Evaluation of Multilingual Models
 BERT (monolingual) with XLM-RoBERTa (multilingual)
on sentiment classification across multiple languages.


---

Author: Gunjan Kumar

Date: 4 November 2025

In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from datasets import load_dataset, Dataset
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print("imports successful!")
print(f"Using device: {torch.device('cuda' if torch.cuda.is_available() else 'cpu')}")


imports successful!
Using device: cpu


In [2]:
def load_multilingual_amazon_reviews():
# Amazon multilingual reviews dataset
    # Load datasets for multiple languages
    languages = ['en', 'de', 'fr', 'es', 'ja']
    language_names = {
        'en': 'English',
        'de': 'German',
        'fr': 'French',
        'es': 'Spanish',
        'ja': 'Japanese'
    }

    datasets = {}

    try:
        # Load Amazon reviews for each language
        for lang in languages:
            print(f"  Loading {language_names[lang]}...")
            ds = load_dataset("amazon_reviews_multi", lang, split='train[:2000]')
            # Map ratings to sentiment: 1-2=negative, 3=neutral, 4-5=positive
            ds = ds.map(lambda x: {
                'text': x['review_body'],
                'label': 0 if x['stars'] <= 2 else (1 if x['stars'] == 3 else 2),
                'language': lang
            })
            datasets[lang] = ds

        print(f"Successfully loaded {len(languages)} languages")
        return datasets, language_names

SyntaxError: incomplete input (ipython-input-632391921.py, line 29)